# Extractive Summary as Text Matching on *FairySum*  🧚

## prove

In [ ]:
import json

data_path="/home/lavallone/Desktop/NUANS/NUANS/Project/match_sum_experiments_data/ACL2020_data/test_CNNDM_bert.jsonl"
data = []
with open(data_path) as f:
    for line in f:
        data.append(json.loads(line))

In [ ]:
for i in range(len(data)):
    print(data[i]["label"])
    print(len(data[i]["indices"]))
    print(len(data[i]["score"]))
    print(len(data[i]["candidate_id"]))
    
    print(len(data[i]["candidate_id"][0]))
    print(len(data[i]["candidate_id"][1]))
    
len(data)

In [ ]:
import os
gold = {}
for file_name in ["FairySum/gold/"+x for x in os.listdir("FairySum/gold/")]:
  key = file_name.split("/")[-1]
  key = "_".join(key.split("_")[:2])
  f = open(file_name, 'r')
  lines = f.readlines()
  text = []
  for l in lines:
    i = l.index(":") + 2
    text.append(l[i:-1])
  if key in gold.keys():
    gold[key] += len(text)
  else:
    gold[key] = len(text)

c=Counter()
c.update([f[:-12] for f in os.listdir("FairySum/gold")])
for k,_ in gold.items():
  gold[k]= round(gold[k]/c[k])

#json.dump(gold, open("data/gold_length.json", "w"))

In [ ]:
# a quanto pare manca un testo in gold 
texts = json.load(open("data/texts.json","r"))
for k in texts.keys():
    if k not in gold.keys():
        print(k)
        
# manca 'bn_02975525n_Rothschild_s Violin'

In [ ]:
!wget https://www.gutenberg.org/files/208/208.txt -O 208_daisy_miller.txt
from booknlp.booknlp import BookNLP
model_params={
		"pipeline":"entity,quote,supersense,event,coref", 
		"model":"big", 
	}

booknlp=BookNLP("en", model_params)
inputFile="208_daisy_miller.txt"
outputDir="208_daisy_miller/"
idd="208_daisy_miller"

booknlp.process(inputFile, outputDir, idd)

## Imports & Downloads

In [ ]:
# install the requirements
%pip install -r requirements.txt > /dev/null
# set to false if you already have the dataset
download_dataset = False 
if download_dataset:
    %cd FairySum
    !bash download_dataset.sh
    %cd ..

In [ ]:
#from src.data_module import MVTec_Dataset, MVTec_DataModule
#from src.AE_simple import AE
#from src.AE_CODE import CODE_AE
#from src.AE_mixer import Mixer_AE
#from src.hyperparameters import Hparams
#from src.train import train_model
from src.hyperparameters import Hparams
from sbert.baseline import SentenceBERT
from sbert.regression_model import execute_booknlp_pipeline
from sbert.regression_model import count_event_sentence
from sbert.regression_model import LengthRegressionModel

import dataclasses
from dataclasses import asdict
import matplotlib.pyplot as plt
import wandb
import pprint
import json
import torchvision
import pytorch_lightning as pl
import gc
from collections import Counter
import seaborn as sns
from tqdm import tqdm
from booknlp.booknlp import BookNLP
import os
import pandas as pd
import numpy as np
from math import comb
import random
# reproducibility stuff
import numpy as np
import random
import torch
np.random.seed(0)
random.seed(0)
torch.cuda.manual_seed(0)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True  # Note that this Deterministic mode can have a performance impact
torch.backends.cudnn.benchmark = False
_ = pl.seed_everything(0)
# to have a better workflow using notebook https://stackoverflow.com/questions/5364050/reloading-submodules-in-ipython
# these commands allow to update the .py codes imported instead of re-importing everything every time.
%load_ext autoreload
%autoreload 2
#%env WANDB_NOTEBOOK_NAME = ./notebook.ipynb
gc.collect()

## Utilities

In [ ]:
# order a dictionary based on its keys in order to manipulate it better
def order_dict(d):
    keys = list(d.keys())
    keys.sort()
    ris = {i: d[i] for i in keys}
    return ris

# selection strategy - candidates creation


## Preprocessing

### Candidates Extraction

**Phase 1** - *output summaries length regression*

In [ ]:
# training the regression model for predicting the best summary length (i.e. the number of sentences to be extracted)

"""
    - The original texts length are saved in 'original_length.json' file. 
    - The golden output lengths (that are the computed average lenghts of the manually annotated summaries by the students) are saved 
      in the 'gold_length.json' file.
      We now need to compute a quantity which quantifies somehow the concentration of events in each story. We leverage the BookNLP library 
      to do so. The adopted strategy is very simple: once the library has detected the events, we count the number of unique sentences that
      contain at least one EVENT (it means that these sentences are relevant for the story).
"""

texts = json.load(open("data/texts.json","r"))

booknlp_already_computed = True if os.path.isdir("data/booknlp_processed_texts")==True else False
if not booknlp_already_computed:
  !python -m spacy download en_core_web_sm > /dev/null # needed for the BookNLP library
  model_params={"pipeline":"entity,event", "model":"big"}
  booknlp = BookNLP("en", model_params)
  texts = json.load(open("data/texts.json","r"))
  execute_booknlp_pipeline(booknlp, texts)
  !rm "data/current_story.txt"
  !rm -r "data/current"
  
events_already_computed = True
if not events_already_computed:
  count_event_sentence("data/booknlp_processed_texts/", texts)

# starting the regression phase
# since we miss one golden summary --> 'bn_02975525n_Rothschild_s Violin', we need to remove it from 'events' and from 'original_lenghts'
events = json.load(open("data/events.json","r"))
del events["bn_02975525n"]
events = order_dict(events)

gold_lengths = json.load(open("data/gold_length.json","r"))
gold_lengths = order_dict(gold_lengths)
original_lenghts = json.load(open("data/original_length.json","r"))
del original_lenghts["bn_02975525n"]
original_lenghts = order_dict(original_lenghts)

# instantiate the regression model for predicting  the output length of our generated extractive summaries
LengthRegressionModel = LengthRegressionModel(gold_lengths, original_lenghts, events)
# LengthRegressionModel.plot() # if we want to plot the regression curve
LengthRegressionModel.fit()
predictions = LengthRegressionModel.predict()


**Phase 2** - *sentences extraction*

In [ ]:
hparams = asdict(Hparams())
hparams["sbert_mode"] = "extraction"
sbert = SentenceBERT(hparams, predictions)

texts = json.load(open("data/texts.json","r"))
extracted_sentences = sbert(texts) # we receive a dictionary with the extracted sentences indices for each story

**Phase 3** - *sentences selection*

In [ ]:
# selection strategy

k_range = hparams["k_range"]
pick_random_n = hparams["pick_random_n"]

candidates_dict = {}
for k,v in extracted_sentences.items():
    candidates_list = []
    candidates_list.append(v)
    n = len(v)
    for i in range(n-k_range, n):
        #c = comb(n, i) # total number of combinations
        random_idx = []
        for _ in range(pick_random_n): # how many random combinations?
            candidates_list.append([v[i] for i in sorted(random.sample(range(n), i))])
            
    candidates_dict[k] = candidates_list
    
# save them
json.dump(candidates_dict, open("data/candidates/candidates.json", "w"))

### Abstractive Summaries

We use  the SOTA **PEGASUS model** for computing the abstractive summaries needed  for  training. We simply download a pretrained model and use it as it is (*plug-and-play*).

In [ ]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

# create the tokenizer
tokenizer = PegasusTokenizer.from_pretrained("google/pegasus-large")

# load the model
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = PegasusForConditionalGeneration.from_pretrained("google/pegasus-large").to(torch_device)

In [ ]:
# we need to know wich are the training texts
train_keys = []
for f in os.listdir("FairySum/texts/train/"):
    k = "_".join(f.split("_")[:2])
    train_keys.append(k)

In [ ]:
texts = json.load(open("data/texts.json","r"))
abstractives = {}

# we make inference on this model
model.eval()
with torch.no_grad():
    for i, (k,v) in tqdm(enumerate(texts.items())):
        if k not in train_keys:
            continue
        # we need to have the text available as a whole
        text = " ".join([e+"\n" for e in v])
        # create tokens batch
        batch = tokenizer.prepare_seq2seq_batch(text, truncation=True, padding="longest", return_tensors="pt").to(torch_device)
        # summary in tokens
        summary = model.generate(**batch)
        # (we need to decode it)
        abstractive_output = tokenizer.batch_decode(summary, skip_special_tokens=True)[0]
        abstractives[k] = abstractive_output
        ##########################
        ####   free GPU RAM   ####
        ##########################
        del batch
        del summary
        del abstractive_output
        torch.cuda.empty_cache()
        ##########################
        
# save the dictionary
json.dump(abstractives, open("data/abstractives/abstractives.json", "w"))

> We use *pegasus-large* pretrainied model because it outputs longer summaries, but since it is trained mainly on news articles which are shorter than our dataset, the output summaries are certainly not ideal. This can be a research direction to follow for future improvements of the overall method. For this mini-project we keep the model as it is, hoping in the achievement of decent results.

### Gold Summaries processing

In [ ]:
# we want to create a dictionary for each gold summary with only the sentences indeces
# of course only of the TRAIN texts
gold = {}
for file_name in ["FairySum/gold/"+x for x in os.listdir("FairySum/gold/")]:
  key = file_name.split("/")[-1]
  key = "_".join(key.split("_")[:2])
  if key not in train_keys:
    continue
  f = open(file_name, 'r')
  lines = f.readlines()
  text = []
  for l in lines:
    i = l.index(":")
    text.append(int(l[:i]))
  if key in gold.keys(): # if we have more than one gold summary we apppend to the list
    gold[key].append(text)
  else:
    gold[key] = [text]
    
# save the dictionary
json.dump(gold, open("data/gold/gold.json", "w"))


### Tokenization ?

## MatchSum

### Finetuning on FairySum

## Evaluation

### ROUGE installation

In [ ]:
## ROUGE installation --> it's been a nightmare! ##
!sudo apt-get install libxml-dom-perl
%cd /content
!git clone https://github.com/andersjo/pyrouge
!cp -r pyrouge/tools/ROUGE-1.5.5 /content/ROUGE
!rm -r pyrouge
!export ROUGE_EVAL_HOME="/content/ROUGE/data/"
%cd /content/ROUGE/data/WordNet-2.0-Exceptions/
!rm  WordNet-2.0.exc.db
!./buildExeptionDB.pl . exc WordNet-2.0.exc.db
%cd ..
!rm  WordNet-2.0.exc.db
!ln -s WordNet-2.0-Exceptions/WordNet-2.0.exc.db WordNet-2.0.exc.db

%cd /content
!git clone https://github.com/bheinzerling/pyrouge
%cd /content/pyrouge
!python setup.py install
!pyrouge_set_rouge_path /content/ROUGE/
#!python -m pyrouge.test

!pip install pyrouge==0.1.3
!pip install rouge==1.0.0

In [ ]:
## CLONING REPO ##
%cd /content
!rm -rf NUANS > /dev/null
!git clone https://github.com/lavallone/NUANS.git
%cd /content/NUANS/Homework_02/MatchSum

In [ ]:
!git pull
!CUDA_VISIBLE_DEVICES=0 python train_matching.py --mode=test --encoder=roberta --save_path=/content/models --gpus=0

In [ ]:
with open("/content/data/test_CNNDM_roberta.jsonl") as json_file:
    l = list(json_file)

l1=[]
for elem in l:
    elem = json.loads(elem)
    l1.append(elem)

l1[0].keys() # ict_keys(['label', 'text', 'summary', 'ext_idx', 'indices', 'score', 'candidate_id', 'text_id', 'summary_id'])
len(l1[0]["text"])
len(l1[0]["ext_idx"]) # sono gli indici delle frasi più salienti 
len(l1[0]["indices"]) # sono l'insieme delle frasi selezionate
len(l1[0]["score"]) # è lo score delle 20 frasi selezionate

# 'candidate_id', 'text_id', 'summary_id' sono le versioni tokenizzate dei candidati, del testo e del golden summary
# label è una sorta di 'id' del sample del dataset